In [ ]:
# !pip install tensorflow==1.15.0

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.metrics import accuracy_score
from tensorflow.contrib.layers import conv2d, avg_pool2d, max_pool2d
from tensorflow.contrib.layers import batch_norm, l2_regularizer
print(tf.__version__)

In [ ]:
import cv2

In [ ]:
# parameters of squeezeNet

img_height = 224
img_width = 224
img_channel = 3
batch_size = 16
max_iter = 4000
lr = 0.01

In [ ]:
num_classes = 10

In [ ]:
#getting train data

train_text_file = pd.read_csv("./annotations/train_labels.csv")

train_images = []
train_labels = []
for i, row in train_text_file.iterrows():
#     print(i)
    if (row['class_id']<10):
        image_name = row['image_name']
        img = cv2.imread(image_name)
        img1 =cv2.resize(img, (img_height, img_width))
        norm_image = cv2.normalize(img1, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
#         print(norm_image)
    #     cv2.imshow("img", img)
    #     cv2.waitKey(100)
        train_images.append(norm_image)
        train_labels.append(row['class_id'])
train_images = np.array(train_images)
train_labels = np.array(train_labels)
train_labels = np.resize(train_labels, (train_labels.shape[0],1))


In [ ]:
#getting test data
test_text_file = pd.read_csv("./annotations/test_labels.csv")
test_images = []
test_labels = []
for i, row in test_text_file.iterrows():
#     print(i)
    if (row['class_id']<10):
        image_name = row['image_name']
        img = cv2.imread(image_name)
        img1 =cv2.resize(img, (img_height, img_width))
        norm_image = cv2.normalize(img1, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

    #     cv2.imshow("img", img)
    #     cv2.waitKey(100)
        test_images.append(norm_image)
        test_labels.append(row['class_id'])
test_images = np.array(test_images)
test_labels = np.array(test_labels)
test_labels = np.resize(test_labels, (test_labels.shape[0],1))


In [ ]:
train_images.shape, train_labels.shape

In [ ]:
test_images.shape, test_labels.shape

In [ ]:
# fire module in squeezeNet

def fire(inputs1, squeeze_op, expand_op1, expand_op2 , scope=None):
    with tf.variable_scope(scope, 'fire'):
        net1 = conv2d(inputs1, squeeze_op, [1, 1], stride=1, scope='squeeze')
        e1 = conv2d(net1, expand_op1, [1, 1], stride=1, scope='1x1')
        e3 = conv2d(net1, expand_op2, [3, 3], scope='3x3')
    return tf.concat([e1, e3], 3)

In [ ]:
# https://github.com/tensorflow/docs/blob/r1.14/site/en/api_docs/python/tf/contrib/layers/conv2d.md

In [ ]:
# defining squeezeNet architecture

def SqueezeNet(inputs, num_classes):
    with tf.variable_scope("squeezenet"):
        net = conv2d(inputs, 96, [7, 7], stride=2, scope='conv1')
        print(net.name, net.get_shape().as_list())
        net = max_pool2d(net, [3, 3], stride=2, scope='maxpool1')
        print(net.name, net.get_shape().as_list())
        net = fire(net, 16, 64, 64, scope='fire2')
        print(net.name, net.get_shape().as_list())
        net = fire(net, 16, 64, 64, scope='fire3')
        print(net.name, net.get_shape().as_list())
        net = fire(net, 32, 128, 128,scope='fire4')
        print(net.name, net.get_shape().as_list())
        net = max_pool2d(net, [3, 3], stride=2, scope='maxpool4')
        print(net.name, net.get_shape().as_list())
        net = fire(net, 32, 128, 128, scope='fire5')
        print(net.name, net.get_shape().as_list())
        net = fire(net, 48, 192, 192, scope='fire6')
        print(net.name, net.get_shape().as_list())
        net = fire(net, 48, 192, 192, scope='fire7')
        print(net.name, net.get_shape().as_list())
        net = fire(net, 64, 256, 256, scope='fire8')
        print(net.name, net.get_shape().as_list())
        net = max_pool2d(net, [3, 3], stride=2, scope='maxpool8')
        print(net.name, net.get_shape().as_list())
        net = fire(net, 64, 256, 256, scope='fire9')
        print(net.name, net.get_shape().as_list())
        net = conv2d(net, num_classes, [1, 1], stride=1, scope='conv10')
        print(net.name, net.get_shape().as_list())
        net_shape = net.get_shape().as_list()
#         net = avg_pool2d(net, [13, 13], stride=1, scope='avgpool10')
        net = avg_pool2d(net, [net_shape[1],net_shape[1]], stride=1, scope='avgpool10')
        print(net.name, net.get_shape().as_list())
        
        net = tf.squeeze(net, [2], name='logits')
        print(net.name, net.get_shape().as_list())
        return net


In [ ]:
# define a function for iterating through mini-batches of the data
def batch_generator(n_iter, X, y,  batch_size=16, 
                    shuffle=False, random_seed=None):
    batch_nos = [i for i in range(0, X.shape[0], batch_size)]
    i = int((n_iter%len(batch_nos)))
#     print("batch : ", i)
    if shuffle:  # shuffles sample order in output batch
        
        i = np.random.choice(batch_nos)
#     print("batch : ", i)
    return X[i:i+batch_size, :], y[i:i+batch_size]

In [ ]:
# defining placeholders

# Initialize placeholders 
x = tf.placeholder(dtype = tf.float32, shape = [None, img_height, img_width, img_channel])
y = tf.placeholder(dtype = tf.int32, shape = [None,1])
# logits = tf.placeholder(dtype = tf.int32, shape = [None,1,1,num_classes])


logits = SqueezeNet(x,num_classes)
# Define a loss function
loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, 
                                                                    logits = logits))
# Define an optimizer 
train_op = tf.train.AdamOptimizer(learning_rate=lr).minimize(loss)


In [ ]:
#training the squeezeNet

tf.set_random_seed(1234)
sess = tf.Session()

sess.run(tf.global_variables_initializer())

for i in range(max_iter):
#         print('EPOCH', i)
        input_images, input_labels = batch_generator(i, train_images, train_labels, batch_size, shuffle=True)
#         print(input_images.shape, input_labels.shape)
        logits1, op, iter_loss = sess.run([logits, train_op, loss], feed_dict={x: input_images, y: input_labels})
    
        if i % 10 == 0:
#             print(input_images.shape)
#             print(np.array(logits1).shape)
            pred = np.argmax(logits1, 2)
#             print("pred : ", pred)
#             print("logits1 : ", logits1)
#             print("actual : ", input_labels)
            acc= accuracy_score(input_labels, pred)
            print("Loss: ", iter_loss, "acc : ", acc)
saver = tf.train.Saver()
saver.save(sess, "./model_ckpt/squeezenet.ckpt")
#         print('DONE WITH EPOCH')


pred_test = []
for i in range(test_images.shape[0]):
    logits1 = sess.run([logits], feed_dict={x: np.resize(test_images[i], (1,img_height, img_width, img_channel))})
#     print(np.array(logits1).shape)
#     print(np.resize(test_images[i], (1,img_height, img_width, img_channel)).shape)
    pred = np.argmax(logits1,3)
    pred_test.append(pred[0][0][0])
acc= accuracy_score(test_labels, pred_test)
print("test accuracy : ", acc)
sess.close()